In [6]:
import pandas as pd
import os
import hashlib
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

filepath = os.path.join(os.getcwd(),"Data")
labeled_dataset = pd.DataFrame.from_csv(os.path.join(filepath,"train.csv"), index_col = None)
"Dataset Size (Rows, Columns): " + str(train_initial.shape)

'Dataset Size (Rows, Columns): (188318, 132)'

In [10]:
labeled_dataset.isnull().sum().sum() #no missing data!

0